# Heroes of Pymoli

In [1]:
# import dependencies
import pandas as pd
import json
import os

In [2]:
jsondata = os.path.join("Resources","purchase_data2.json")

In [3]:
#Read JSON data into a variable
with open(jsondata) as json_data:
    d = json.load(json_data)

In [4]:
#turn data into dataframe
game_df = pd.DataFrame(d, columns=['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'])
game_df.head()

,SN,Age,Gender,Item ID,Item Name,Price
0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49
1,Aidaira26,21,Male,12,Dawne,3.36
2,Irim47,17,Male,5,Putrid Fan,2.63
3,Irith83,17,Male,123,Twilight's Carver,2.55
4,Philodil43,22,Male,154,Feral Katana,4.11


In [5]:
#Player Count
total_players = len(game_df['SN'].value_counts())
Total_Players = pd.DataFrame({"Total Players": total_players}, index=[0])
Total_Players

,Total Players
0,74


# Purchasing Analysis

In [6]:
#Number of Unique Items
unique_items = len(game_df['Item ID'].value_counts())

#Average Purchase Price
average_price = game_df['Price'].mean()

#Total Number of Purchases
total_purchases = game_df['Item Name'].count()


#Total Revenue
total_revenue = game_df['Price'].sum()

#Create DataFrame
purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items],
                                   "Average Price": [average_price],
                                   "Total Purchases": [total_purchases],
                                   "Total Revenue": [total_revenue],
                                
})

#Reorder DataFrame
purchasing_analysis = purchasing_analysis[["Number of Unique Items", "Average Price","Total Purchases", "Total Revenue"]]
                                

#improve formatting
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${0:,.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${0:,.2f}".format)

#Reorder Columns
purchasing_analysis

,Number of Unique Items,Average Price,Total Purchases,Total Revenue
0,64,$2.92,78,$228.10


# Gender Demographics

In [7]:
#Group data by Gender and filter duplicates
grouped_df = game_df.groupby(["Gender"])
unique_df = grouped_df.nunique()

#Total Gender
total_gender = unique_df["SN"].sum()

#Percentage and Count of Players
count = unique_df["SN"].unique()
percentage = unique_df["SN"]/ total_gender

#Create new dataframe
final_gender = pd.DataFrame({"Percentage of Players": percentage,
                            "Count":count})
#Change percentage format and re order columns
final_gender["Percentage of Players"] = final_gender["Percentage of Players"].map("{:,.2%}".format) 
final_gender.columns = ["Percentage of Players", "Count"]
#Print final dataframe
final_gender

,Percentage of Players,Count
Gender,,
Female,13,17.57%
Male,60,81.08%
Other / Non-Disclosed,1,1.35%


# Purchasing Analysis (Gender)

In [8]:
#Purchase Count
purchase_count = unique_df["Gender"].value_counts()

#Average Purchase Price
average_price = grouped_df["Price"].mean()

#Total Purchase Value
purchase_price = grouped_df["Price"].sum()

#Normalized Totals
normalized = purchase_price / count

#Create new dataframe
gender_analysis = pd.DataFrame({"Average Purchase Price": average_price,"Total Purchase Price":purchase_price,"Normalized Totals": normalized})

#Clean up formatting and reorder columns
gender_analysis["Average Purchase Price"] = gender_analysis["Average Purchase Price"].map("${:,.2f}".format) 
gender_analysis["Total Purchase Price"] = gender_analysis["Total Purchase Price"].map("${:,.2f}".format) 
gender_analysis["Normalized Totals"] = gender_analysis["Normalized Totals"].map("${:,.2f}".format) 
#Reorder Columns
gender_analysis = gender_analysis[["Average Purchase Price", "Total Purchase Price", "Normalized Totals"]]

gender_analysis.head()

,Average Purchase Price,Total Purchase Price,Normalized Totals
Gender,,,
Female,$3.18,$41.38,$3.18
Male,$2.88,$184.60,$3.08
Other / Non-Disclosed,$2.12,$2.12,$2.12


# Age Demographics

In [9]:
#Age Demographics
#Drop Duplicates
cleaned_df = game_df.drop_duplicates("SN")

#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
bins = [10, 14, 18, 22, 26, 30, 34, 38, 42]
groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Create a  new column for age groups and then groupby Age Groups
cleaned_df["Age Groups"] = pd.cut(cleaned_df["Age"], bins, labels=groups)
age_df = cleaned_df.groupby(["Age Groups"])

total_age = unique_df["Age"].sum()

#Purchase Count
age_purchase = cleaned_df["Age Groups"].value_counts()

#Percentage of Users
age_percentage = age_purchase / total_players

age_demographics = pd.DataFrame({"Total Count": age_purchase,
                             "Percentage of Players":age_percentage})

age_demographics["Percentage of Players"] = age_demographics["Percentage of Players"].map("{:,.2%}".format) 

age_demographics = pd.concat([age_demographics.loc[["<10"],:], age_demographics.drop("<10", axis=0)], axis=0)

age_demographics

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Percentage of Players,Total Count
<10,4.05%,3
20-24,28.38%,21
15-19,25.68%,19
10-14,14.86%,11
35-39,6.76%,5
30-34,5.41%,4
25-29,5.41%,4
40+,2.70%,2


# Age Analysis

In [14]:
#Average Purchase Price
age_average_price = age_df["Price"].mean()

#Total Purchase Value
age_price = age_df["Price"].sum()

#Normalized Totals
normalized_age = age_price / age_purchase

#Create new dataframe
age_analysis = pd.DataFrame({"Purchase Count": age_purchase,
                             "Average Purchase Price":age_average_price,
                            "Total Purchase Value":age_price,
                            "Normalized Totals": normalized_age})

#Clean up formatting
age_analysis["Average Purchase Price"] = age_analysis["Average Purchase Price"].map("${:,.2f}".format) 
age_analysis["Total Purchase Value"] = age_analysis["Total Purchase Value"].map("${:,.2f}".format) 
age_analysis["Normalized Totals"] = age_analysis["Normalized Totals"].map("${:,.2f}".format) 

#Reorder Columns
age_analysis = age_analysis[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]

#Move bottom row to the top
age_analysis = pd.concat([age_analysis.loc[["<10"],:], age_analysis.drop("<10", axis=0)], axis=0)

age_analysis


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
<10,3,$2.99,$8.96,$2.99
10-14,11,$2.76,$30.41,$2.76
15-19,19,$3.08,$58.57,$3.08
20-24,21,$3.00,$62.99,$3.00
25-29,4,$2.69,$10.77,$2.69
30-34,4,$1.89,$7.54,$1.89
35-39,5,$3.74,$18.72,$3.74
40+,2,$3.65,$7.30,$3.65


# Top Users

In [11]:
grouped_sn = game_df.groupby(["SN"])

#Find total spent per user
total_price_sn = grouped_sn.sum()["Price"]

#Find avg spent per user
avg_price_sn = grouped_sn.mean()["Price"]

#Find purchase count per user
count_sn = grouped_sn.count()["Price"]

#Create new dataframe
top_user_df = pd.DataFrame({"Purchase Count":count_sn,
                            "Average Purchase Price":avg_price_sn,
                            "Total Purchase Price": total_price_sn
                            })

#Sort by total purchase price
sorted_df = top_user_df.sort_values("Total Purchase Price",ascending=False)

#Format numbers
sorted_df["Average Purchase Price"] = sorted_df["Average Purchase Price"].map("${:,.2f}".format) 
sorted_df["Total Purchase Price"] = sorted_df["Total Purchase Price"].map("${:,.2f}".format) 

#Reorder Columns
sorted_df = sorted_df[["Purchase Count", "Average Purchase Price", "Total Purchase Price"]]

#Display top 5
sorted_df.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Sundaky74,2,$3.71,$7.41
Aidaira26,2,$2.56,$5.13
Eusty71,1,$4.81,$4.81
Chanirra64,1,$4.78,$4.78
Alarap40,1,$4.71,$4.71


# Most Popular Items

In [12]:
grouped_id = game_df.set_index(["Item ID", "Item Name"])

grouped_id = grouped_id.groupby(level=["Item ID", "Item Name"])

#Find total spent per user
total_price_id = grouped_id.sum()["Price"]

#Find avg spent per user
avg_price_id = grouped_id.mean()["Price"]

#Find purchase count per user
count_id = grouped_id.count()["Price"]


#Create new dataframe
items_df = pd.DataFrame({ 
                         "Count":count_id,
                            "Average Purchase Price":avg_price_id,
                            "Total Purchase Price": total_price_id,
                            })


#Sort by total purchase price
sorted_items = items_df.sort_values("Count",ascending=False)

sorted_items["Average Purchase Price"] = sorted_items["Average Purchase Price"].map("${:,.2f}".format) 
sorted_items["Total Purchase Price"] = sorted_items["Total Purchase Price"].map("${:,.2f}".format) 

#Reorder Columns
sorted_items = sorted_items[["Count", "Average Purchase Price", "Total Purchase Price"]]


#Display top 5
sorted_items.head(5)



,,Count,Average Purchase Price,Total Purchase Price
Item ID,Item Name,,,
94,Mourning Blade,3,$3.64,$10.92
90,Betrayer,2,$4.12,$8.24
111,Misery's End,2,$1.79,$3.58
64,Fusion Pummel,2,$2.42,$4.84
154,Feral Katana,2,$4.11,$8.22


# Most Profitable Items

In [13]:

grouped_id = game_df.set_index(["Item ID", "Item Name"])

grouped_id = grouped_id.groupby(level=["Item ID", "Item Name"])

#Find total spent per user
total_price_id = grouped_id.sum()["Price"]

#Find avg spent per user
avg_price_id = grouped_id.mean()["Price"]

#Find purchase count per user
count_id = grouped_id.count()["Price"]

#Create new dataframe
items_df = pd.DataFrame({ "Count":count_id,
                        "Average Purchase Price":avg_price_id,
                        "Total Purchase Price": total_price_id,
                            })


#Sort by total purchase price
sorted_items = items_df.sort_values("Total Purchase Price",ascending=False)

sorted_items["Average Purchase Price"] = sorted_items["Average Purchase Price"].map("${:,.2f}".format) 
sorted_items["Total Purchase Price"] = sorted_items["Total Purchase Price"].map("${:,.2f}".format) 

#Reorder Columns
sorted_items = sorted_items[["Count", "Average Purchase Price", "Total Purchase Price"]]

#Display top 5
sorted_items.head(5)


,,Count,Average Purchase Price,Total Purchase Price
Item ID,Item Name,,,
94,Mourning Blade,3,$3.64,$10.92
117,"Heartstriker, Legacy of the Light",2,$4.71,$9.42
93,Apocalyptic Battlescythe,2,$4.49,$8.98
90,Betrayer,2,$4.12,$8.24
154,Feral Katana,2,$4.11,$8.22


# Analyses conducted using dataset 1.
1. Males spent 130% more than females on in app purchases and 192 percent more than other/nondisclosed. 
However, there is only a 13 cent difference in average price spent between male and female, while the average price 
spent from gender non-diclosed is 30 cents more than males.

2. Teenagers are the majority players of the game, at 30%, while 20-24 year olds make up 26 percent. There is a 19 percent
drop in players in the next age group, as 25-29 is 7.7%. 20-24 year olds spend more on average, than 15-19 year olds, while 
30-34 year olds spend the most at $3.35 per transaction.

3. The most popular items does not make the game as much money as the most profitable ones. However, 
the Retribution Axe, which is the most profitable item ties for third on the 
most popular, with a count of nine.  
